<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_04_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pobieramy dane z github wykorzystując funkcję `download.file`

In [1]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")

Musimy dane wczytać do R

In [2]:
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE)
head(dane)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA


Załadujmy pakiet `tidyverse` do przetwarzania danych.

In [ ]:
library(tidyverse)

Porównamy sobie rozkłady liczby ofert pracy w popycie oraz CBOP według wielkości przedsiębiorstwa.

In [9]:
dane %>%
  ## wybieram dane z popytu
  filter(!is.na(id_popyt)) %>%
  ## wyliczam ile było wolnych miejsc pracy wykorzystując funkcję count i wagę
  count(klasa_pr, wt = waga*wolne_miejsca) %>%
  ## obliczam odsetek
  mutate(odsetek = n / sum(n)*100,
         zrodlo = "popyt") -> dane_wielk_popyt

dane %>%
  ## wybieram dane z CBOP
  filter(!is.na(id_cbop)) %>%
  ## obliczam ile było wolnych miejsc pracy
  count(klasa_pr, wt = wolne_miejsca_cbop) %>%
  ## obliczam odsetek
  mutate(odsetek = n / sum(n)*100,
         zrodlo = "cbop") -> dane_wielk_cbop

Porównuje odsetki między dwoma zbiorami danych

In [14]:
dane_wielk_popyt %>%
  select(-n) %>% 
  bind_rows(dane_wielk_cbop %>% select(-n)) %>%
  spread(zrodlo, odsetek)

klasa_pr,cbop,popyt
<chr>,<dbl>,<dbl>
D,51.97230,31.98298
M,23.79764,43.81333
S,24.23007,24.20369


Utwórzmy wartości globalne, które wykorzystamy do post-stratyfikacji. Na tę chwilę wykorzystamy wyłącznie informacje o wielkości podmiotu.

In [18]:
dane_wielk_popyt %>%
  select(klasa_pr, pop = n) -> wartosci_globalne

wartosci_globalne

klasa_pr,pop
<chr>,<int>
D,94124
M,128940
S,71230


Dodajemy do zbioru danych z centralnej bazy ofert pracy informacje o wartościach globalnych z popytu.

In [28]:
dane %>% 
  ## wybieram dane z cbop
  filter(!is.na(id_cbop)) %>%
  ## ograniczam do wybranych kolumn
  select(id_cbop, klasa_pr, jedna_zmiana, wymiar_40, wolne_miejsca_cbop) -> zbior_cbop


zbior_cbop %>%
  ## tworzę podsumowanie według klasy wielkości
  group_by(klasa_pr) %>%
  ## obliczam średnie i liczebności w danym przekroju
  summarise(zmiana = mean(jedna_zmiana),
            n = sum(wolne_miejsca_cbop)) %>%
  ## dodaje wartości globalne z popytu
  left_join(wartosci_globalne) %>%
  ## obliczam wagę post-stratyfikacyjną
  mutate(waga = pop / n) %>%
  ## wyliczam dwa estymatory
  summarise(y_naive = weighted.mean(zmiana, n),
            y_ps = weighted.mean(zmiana, waga))

Joining, by = "klasa_pr"



y_naive,y_ps
<dbl>,<dbl>
0.6266626,0.7064367


Komentarz do uzyskanych wyników:


1.   Estymator naiwny (`y_naive`) obliczony wyłącznie na podstawie danych z CBOP informuje, że 63% podmiotów miało wakaty, które są na na jedną zmianę.
2.   Estymator post-stratyfikcyjny (`y_ps`) obliczony na danych z CBOP ale uwzględniajac różnicę w rozkłdach ze względu na wielkość firmy informuje, że (prawdopodobinie) odsetek ten w populacji jest wyższy i wynosci 71%.

Jeżeli widzą Państwo różnicę w wartościach miedzy `y_naive`, a `y_ps` to oznacza, że zmienna wykorzystana do post-stratyfikacji działa (bo różnicuje populację). Można to sprawdzić licząc wcześniej współczynnik korelacji V-Cramer'a na podstawie statystyki $\chi^2$.

